In [4]:
# =========================================================
# 1️⃣ LIBRERÍAS
# =========================================================
!pip install -q python-docx

import pandas as pd
import locale
from pathlib import Path
from docx import Document
from docx.shared import RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.table import WD_ALIGN_VERTICAL
from docx.oxml import OxmlElement
from docx.oxml.ns import qn

try:
    locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
except:
    locale.setlocale(locale.LC_ALL, '')

In [5]:
# =========================================================
# 2️⃣ FUNCIONES REUTILIZABLES
# =========================================================

def cargar_datos(ruta, hoja):
    return pd.read_excel(ruta, sheet_name=hoja)

def exportar_tabla_word(df, titulo, nombre_archivo,
                         columnas_numericas=None,
                         filas_negrilla=None,
                         filas_sombreado=None):

    columnas_numericas = columnas_numericas or []
    filas_negrilla = filas_negrilla or []
    filas_sombreado = filas_sombreado or []

    doc = Document()
    doc.add_heading(titulo, level=1)

    table = doc.add_table(rows=len(df)+1, cols=len(df.columns))

    # Encabezados
    for i, col in enumerate(df.columns):
        cell = table.rows[0].cells[i]
        cell.text = str(col)
        p = cell.paragraphs[0]
        run = p.runs[0]
        run.bold = True
        run.font.color.rgb = RGBColor(255,255,255)
        p.alignment = WD_ALIGN_PARAGRAPH.CENTER
        cell.vertical_alignment = WD_ALIGN_VERTICAL.CENTER
        shading = OxmlElement('w:shd')
        shading.set(qn('w:fill'),'000080')
        cell._tc.get_or_add_tcPr().append(shading)

    # Filas
    for r, row in enumerate(df.itertuples(index=False),1):
        for c, val in enumerate(row):
            cell = table.rows[r].cells[c]
            p = cell.paragraphs[0]
            run = p.runs[0] if p.runs else p.add_run()

            if df.columns[c] in columnas_numericas:
                run.text = locale.format_string("%d", int(val), grouping=True)
                p.alignment = WD_ALIGN_PARAGRAPH.RIGHT
            else:
                run.text = str(val)
                p.alignment = WD_ALIGN_PARAGRAPH.LEFT

            if any(df.iloc[r-1].astype(str).str.contains(f, na=False).any() for f in filas_negrilla):
                run.bold = True

            if any(df.iloc[r-1].astype(str).str.contains(f, na=False).any() for f in filas_sombreado):
                shading = OxmlElement('w:shd')
                shading.set(qn('w:fill'),'D9E1F2')
                cell._tc.get_or_add_tcPr().append(shading)

    output = f'/content/{nombre_archivo}'
    doc.save(output)
    print(f"✅ Archivo generado: {output}")

In [6]:
# =========================================================
# 3️⃣ CARGA Y FILTRO
# =========================================================

file_path = '/content/Plantilla Reporte TuCatastro.xlsx'
df = cargar_datos(file_path, 'CRUDOS')

municipio = 'APULO'
df = df[df['NOM_MUN']==municipio].copy()

df = df[df['RAD_AÑO'].isin([2024,2025])]

In [7]:
# =========================================================
# 4️⃣ TABLA 1
# =========================================================

tabla1 = (
    df.groupby(['Estado','RAD_AÑO'])['Numero radicado']
      .nunique()
      .unstack(fill_value=0)
)

tabla1['Total general'] = tabla1.sum(axis=1)
tabla1.loc['Total general'] = tabla1.sum()
tabla1 = tabla1.reset_index().rename(columns={'Estado':'Estado de la radicacion'})
tabla1[[2024,2025,'Total general']] = tabla1[[2024,2025,'Total general']].astype(int)

exportar_tabla_word(
    tabla1,
    'Tabla 1. Clasificación de los radicados',
    'Reporte_Anual_Tramites-Tabla1.docx',
    columnas_numericas=[2024,2025,'Total general'],
    filas_negrilla=['Total general']
)

✅ Archivo generado: /content/Reporte_Anual_Tramites-Tabla1.docx


In [8]:
# =========================================================
# 5️⃣ TABLA 2
# =========================================================

tabla2 = (
    df.groupby(['OFICINA DE GESTION','Tipo','RAD_AÑO'])['Numero radicado']
      .nunique()
      .unstack(fill_value=0)
)

tabla2['Total general'] = tabla2.sum(axis=1)

# Subtotales por oficina
subtotal = tabla2.groupby(level=0).sum()
subtotal['Tipo']='Subtotal'
subtotal = subtotal.set_index('Tipo',append=True)

tabla2 = pd.concat([tabla2,subtotal]).sort_index()

# Total general
total_general = pd.DataFrame(tabla2.sum()).T
total_general.index = pd.MultiIndex.from_tuples([('Total general','')])
tabla2 = pd.concat([tabla2,total_general]).reset_index()

tabla2[[2024,2025,'Total general']] = tabla2[[2024,2025,'Total general']].astype(int)

exportar_tabla_word(
    tabla2,
    'Tabla 2. Radicados por tipo y quien resuelve',
    'Reporte_Anual_Tramites-Tabla2.docx',
    columnas_numericas=[2024,2025,'Total general'],
    filas_negrilla=['Subtotal','Total general']
)

✅ Archivo generado: /content/Reporte_Anual_Tramites-Tabla2.docx


In [9]:
# =========================================================
# 6️⃣ TABLA 3 (antes tabla 4)
# =========================================================

df['RES_AÑO_GROUP'] = pd.Series(
    pd.NA,index=df.index
)

df.loc[df['Fecha de Resolución'].isna(),'RES_AÑO_GROUP']='En proceso'
df.loc[df['RES_AÑO']==2024,'RES_AÑO_GROUP']='Resolución 2024'
df.loc[df['RES_AÑO']==2025,'RES_AÑO_GROUP']='Resolución 2025'
df['RES_AÑO_GROUP']=df['RES_AÑO_GROUP'].fillna('Otros')

tabla3 = (
    df.groupby(['RAD_AÑO','OFICINA DE GESTION','Tipo','RES_AÑO_GROUP'])['Numero radicado']
      .nunique()
      .unstack(fill_value=0)
)

tabla3['Total general']=tabla3.sum(axis=1)

# Subtotales jerárquicos
subtotal_oficina = tabla3.groupby(level=[0,1]).sum()
subtotal_oficina['Tipo']='Subtotal'
subtotal_oficina = subtotal_oficina.set_index('Tipo',append=True)

subtotal_anio = tabla3.groupby(level=0).sum()
subtotal_anio['OFICINA DE GESTION']='Total Año'
subtotal_anio = subtotal_anio.set_index('OFICINA DE GESTION',append=True)

tabla3 = pd.concat([tabla3,subtotal_oficina,subtotal_anio])

# Total general final
grand_total = pd.DataFrame(tabla3.sum()).T
grand_total.index=pd.MultiIndex.from_tuples([('Total general','','')])
tabla3 = pd.concat([tabla3,grand_total]).reset_index()

tabla3[['En proceso','Resolución 2024','Resolución 2025','Total general']] = \
tabla3[['En proceso','Resolución 2024','Resolución 2025','Total general']].astype(int)

exportar_tabla_word(
    tabla3,
    'Tabla 3. Radicados resueltos por año y oficina',
    'Reporte_Anual_Tramites-Tabla3.docx',
    columnas_numericas=['En proceso','Resolución 2024','Resolución 2025','Total general'],
    filas_negrilla=['Subtotal','Total general'],
    filas_sombreado=['Total Año','Total general']
)

✅ Archivo generado: /content/Reporte_Anual_Tramites-Tabla3.docx


In [ ]:
# =========================================================
# 1️⃣ LIBRERÍAS
# =========================================================
from docx import Document
from docx.shared import RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
import pandas as pd
import locale

try:
    locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
except:
    locale.setlocale(locale.LC_ALL, '')

In [ ]:
# =========================================================
# 2️⃣ CÁLCULOS AUTOMÁTICOS DESDE TABLAS
# =========================================================

municipio = "APULO"

# --- TABLA 1 ---
total_radicados = int(tabla1.loc[tabla1['Estado de la radicacion']=='Total general','Total general'].iloc[0])

# porcentaje finalizados (todo lo que no sea "radicado")
finalizados = tabla1[
    (tabla1['Estado de la radicacion']!='radicado') &
    (tabla1['Estado de la radicacion']!='Total general')
]['Total general'].sum()

porcentaje_finalizados = round((finalizados/total_radicados)*100,1)

# --- TABLA 2 ---
total_tabla2 = tabla2.loc[tabla2['level_0']=='Total general','Total general'].iloc[0]

# Suponiendo que enlace territorial es diferente de "Nivel Central"
territorial = tabla2[
    (~tabla2['level_0'].isin(['Total general'])) &
    (~tabla2['level_0'].str.contains('Central',na=False))
]['Total general'].sum()

porcentaje_territorial = round((territorial/total_tabla2)*100,1)
porcentaje_central = round(100 - porcentaje_territorial,1)

In [ ]:
# =========================================================
# 3️⃣ CREAR DOCUMENTO FINAL
# =========================================================

doc_final = Document()

doc_final.add_heading(f"Reporte Anual de Trámites - {municipio}", level=1)

# ------------------ PÁRRAFO 1 ------------------

parrafo1 = (
f"La siguiente tabla corresponde al estado de los radicados por año y con un total general. "
f"En {municipio} se presenta un total de {locale.format_string('%d', total_radicados, grouping=True)} "
f"radicados entre los años 2024 y 2025. "
f"Para estos dos años se han finalizado el {porcentaje_finalizados}% de los trámites recibidos."
)

doc_final.add_paragraph(parrafo1)

# Insertar Tabla 1
doc_final.add_heading("Tabla 1", level=2)
doc_final.add_paragraph("Clasificación de los radicados")
doc_final._body._element.append(Document('/content/Reporte_Anual_Tramites-Tabla1.docx').tables[0]._element)

# ------------------ PÁRRAFO 2 ------------------

parrafo2 = (
f"De los {locale.format_string('%d', total_radicados, grouping=True)} radicados del municipio de {municipio} "
f"el {porcentaje_territorial}% representan trámites que el enlace territorial de la ACC resuelve en el municipio "
f"y el {porcentaje_central}% restante se resuelve a nivel central."
)

doc_final.add_paragraph(parrafo2)

# Insertar Tabla 2
doc_final.add_heading("Tabla 2", level=2)
doc_final.add_paragraph("Radicados por tipo y quien resuelve")
doc_final._body._element.append(Document('/content/Reporte_Anual_Tramites-Tabla2.docx').tables[0]._element)

# ------------------ TABLA 3 ------------------

doc_final.add_heading("Tabla 3", level=2)
doc_final.add_paragraph("Radicados resueltos desagregados por año y oficina")
doc_final._body._element.append(Document('/content/Reporte_Anual_Tramites-Tabla3.docx').tables[0]._element)



In [ ]:
# =========================================================
# 4️⃣ GUARDAR DOCUMENTO FINAL
# =========================================================

ruta_salida = "/content/Reporte_Final_Actualizado.docx"
doc_final.save(ruta_salida)

print("✅ Reporte final generado correctamente:")
print(ruta_salida)

✅ Reporte final generado correctamente:
/content/Reporte_Final_Actualizado.docx


---

In [ ]:
# =========================================================
# 1️⃣ LIBRERÍAS
# =========================================================
!pip install -q python-docx reportlab matplotlib

import pandas as pd
import os
import matplotlib.pyplot as plt
from docx import Document
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import ParagraphStyle
from reportlab.lib import colors
from reportlab.lib.units import inch
from reportlab.platypus import Table
from reportlab.lib.pagesizes import letter
from reportlab.platypus import Image as RLImage
from pathlib import Path
import locale

try:
    locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
except:
    locale.setlocale(locale.LC_ALL, '')



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 40.1 MB/s eta 0:00:00


In [ ]:
# =========================================================
# 2️⃣ CONFIGURACIÓN GENERAL
# =========================================================

BASE_EXCEL = "/content/Plantilla Reporte TuCatastro.xlsx"
PLANTILLA_WORD = "/content/Apulo_Informe tramites Catastral ACC.docx"


ANIOS = [2024, 2025]
CARPETA_RAIZ = "/content/Reportes_ACC_2024_2025"



In [ ]:
# =========================================================
# 3️⃣ FUNCIONES DE TABLAS
# =========================================================

def construir_tabla1(df):
    t = (
        df.groupby(['Estado','RAD_AÑO'])['Numero radicado']
          .nunique()
          .unstack(fill_value=0)
    )
    t['Total general'] = t.sum(axis=1)
    t.loc['Total general'] = t.sum()
    t = t.reset_index().rename(columns={'Estado':'Estado de la radicación'})
    return t

def construir_tabla2(df):
    t = (
        df.groupby(['OFICINA DE GESTION','Tipo','RAD_AÑO'])['Numero radicado']
          .nunique()
          .unstack(fill_value=0)
    )
    t['Total general'] = t.sum(axis=1)
    t = t.reset_index()
    return t

def construir_tabla3(df):
    df['RES_AÑO_GROUP'] = 'En proceso'
    df.loc[df['RES_AÑO']==2024,'RES_AÑO_GROUP']='Resolución 2024'
    df.loc[df['RES_AÑO']==2025,'RES_AÑO_GROUP']='Resolución 2025'

    t = (
        df.groupby(['RAD_AÑO','OFICINA DE GESTION','Tipo','RES_AÑO_GROUP'])
          ['Numero radicado']
          .nunique()
          .unstack(fill_value=0)
    )
    t['Total general']=t.sum(axis=1)
    t = t.reset_index()
    return t



In [ ]:
# =========================================================
# 4️⃣ FUNCIONES DE GRÁFICAS
# =========================================================

def graficar_mes(df, ruta):
    df['MES'] = pd.to_datetime(df['Fecha de Radicación']).dt.month
    t = df.groupby('MES')['Numero radicado'].nunique()
    plt.figure()
    t.plot(kind='bar')
    plt.title("Radicados por Mes")
    plt.xlabel("Mes")
    plt.ylabel("Cantidad")
    plt.tight_layout()
    plt.savefig(ruta)
    plt.close()

def graficar_dia(df, ruta):
    df['DIA'] = pd.to_datetime(df['Fecha de Radicación']).dt.day_name()
    t = df.groupby('DIA')['Numero radicado'].nunique()
    plt.figure()
    t.plot(kind='bar')
    plt.title("Radicados por Día")
    plt.xlabel("Día")
    plt.ylabel("Cantidad")
    plt.tight_layout()
    plt.savefig(ruta)
    plt.close()



In [ ]:
# =========================================================
# 5️⃣ GENERADOR INSTITUCIONAL
# =========================================================

def generar_sistema_reportes(municipio=None):

    df = pd.read_excel(BASE_EXCEL, sheet_name='CRUDOS')
    df = df[df['RAD_AÑO'].isin(ANIOS)]

    if municipio:
        municipios = [municipio]
    else:
        municipios = df['NOM_MUN'].unique()

    os.makedirs(CARPETA_RAIZ, exist_ok=True)

    for mun in municipios:

        df_mun = df[df['NOM_MUN']==mun].copy()
        if df_mun.empty:
            continue

        # Crear estructura carpetas
        base_mun = f"{CARPETA_RAIZ}/{mun}"
        os.makedirs(f"{base_mun}/01_Word", exist_ok=True)
        os.makedirs(f"{base_mun}/02_PDF", exist_ok=True)
        os.makedirs(f"{base_mun}/03_Graficas", exist_ok=True)

        # Construir tablas
        tabla1 = construir_tabla1(df_mun)
        tabla2 = construir_tabla2(df_mun)
        tabla3 = construir_tabla3(df_mun)

        # Graficas
        ruta_mes = f"{base_mun}/03_Graficas/Radicados_por_mes.png"
        ruta_dia = f"{base_mun}/03_Graficas/Radicados_por_dia.png"

        graficar_mes(df_mun, ruta_mes)
        graficar_dia(df_mun, ruta_dia)

        # WORD
        doc = Document(PLANTILLA_WORD)

        for p in doc.paragraphs:
            if "APULO" in p.text:
                p.text = p.text.replace("APULO", mun.upper())

        word_path = f"{base_mun}/01_Word/{mun}_Informe_ACC_2024_2025.docx"
        doc.save(word_path)

        # PDF
        pdf_path = f"{base_mun}/02_PDF/{mun}_Informe_ACC_2024_2025.pdf"
        pdf = SimpleDocTemplate(pdf_path, pagesize=letter)
        elements = []

        elements.append(Paragraph(f"Informe ACC - {mun}", ParagraphStyle('Normal')))
        elements.append(Spacer(1, 0.5*inch))

        elements.append(RLImage(ruta_mes, width=5*inch, height=3*inch))
        elements.append(Spacer(1, 0.5*inch))
        elements.append(RLImage(ruta_dia, width=5*inch, height=3*inch))

        pdf.build(elements)

        print(f"✅ Sistema generado para {mun}")



In [ ]:
# =========================================================
# 6️⃣ EJECUCIÓN
# =========================================================

# 🔹 Municipio específico
generar_sistema_reportes("APULO")

# 🔹 Todos los municipios
# generar_sistema_reportes()


KeyError: 'Fecha de Radicación'

In [ ]:
# =========================================================
# 1️⃣ LIBRERÍAS
# =========================================================
!pip install -q python-docx reportlab matplotlib

import pandas as pd
import os
import matplotlib.pyplot as plt
from docx import Document
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image as RLImage
from reportlab.lib.styles import ParagraphStyle
from reportlab.lib.units import inch
from reportlab.lib.pagesizes import letter
import locale

try:
    locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
except:
    locale.setlocale(locale.LC_ALL, '')

# =========================================================
# 2️⃣ CONFIGURACIÓN GENERAL
# =========================================================

BASE_EXCEL = "/content/Plantilla Reporte TuCatastro.xlsx"
PLANTILLA_WORD = "/content/Apulo_Informe tramites Catastral ACC.docx"

ANIOS = [2024, 2025]
CARPETA_RAIZ = "/content/Reportes_ACC_2024_2025"

# =========================================================
# 3️⃣ NORMALIZACIÓN DE COLUMNAS
# =========================================================

def normalizar_columnas(df):
    df.columns = df.columns.str.strip()
    df.columns = (
        df.columns
        .str.normalize('NFKD')
        .str.encode('ascii', errors='ignore')
        .str.decode('utf-8')
    )
    return df

# =========================================================
# 4️⃣ DETECCIÓN AUTOMÁTICA DE COLUMNA FECHA
# =========================================================

def detectar_columna_fecha(df):
    posibles = [c for c in df.columns if 'fecha' in c.lower() and 'rad' in c.lower()]
    if not posibles:
        raise ValueError("❌ No se encontró columna de fecha de radicación en el Excel.")
    return posibles[0]

# =========================================================
# 5️⃣ FUNCIONES DE TABLAS
# =========================================================

def construir_tabla1(df):

    t = (
        df.groupby(['Estado','RAD_ANO'])['Numero radicado']
          .nunique()
          .unstack(fill_value=0)
          .reindex(columns=[2024, 2025], fill_value=0)
    )

    # Total horizontal
    t['Total general'] = t.sum(axis=1)

    # Total vertical
    t.loc['Total general'] = t.sum()

    t = (
        t.reset_index()
         .rename(columns={
             'Estado': 'Estado de la radicación',
             2024: '2024',
             2025: '2025'
         })
    )

    return t


def construir_tabla2(df):

    t = (
        df.groupby(['OFICINA DE GESTION','Tipo','RAD_ANO'])['Numero radicado']
          .nunique()
          .unstack(fill_value=0)
          .reindex(columns=[2024, 2025], fill_value=0)
    )

    # Total horizontal (por fila)
    t['Total general'] = t.sum(axis=1)

    # Reset limpio
    t = t.reset_index()

    # Renombrar columnas
    t.columns = [
        'Oficina de Gestión',
        'Tipo de trámite',
        '2024',
        '2025',
        'Total general'
    ]

    # ✅ TOTAL VERTICAL (fila final)
    total_row = {
        'Oficina de Gestión': 'Total general',
        'Tipo de trámite': '',
        '2024': t['2024'].sum(),
        '2025': t['2025'].sum(),
        'Total general': t['Total general'].sum()
    }

    t = pd.concat([t, pd.DataFrame([total_row])], ignore_index=True)

    return t


def construir_tabla3(df):

    df = df.copy()

    df['RES_ANO_GROUP'] = 'En proceso'
    df.loc[df['RES_ANO'] == 2024, 'RES_ANO_GROUP'] = 'Resolución 2024'
    df.loc[df['RES_ANO'] == 2025, 'RES_ANO_GROUP'] = 'Resolución 2025'

    t = (
        df.groupby(['RAD_ANO','OFICINA DE GESTION','Tipo','RES_ANO_GROUP'])
          ['Numero radicado']
          .nunique()
          .unstack(fill_value=0)
          .reindex(columns=['En proceso','Resolución 2024','Resolución 2025'], fill_value=0)
    )

    # Total horizontal
    t['Total general'] = t.sum(axis=1)

    # Reset limpio
    t = t.reset_index()

    # Renombrar columnas
    t.columns = [
        'Año',
        'Oficina de Gestión',
        'Tipo de trámite',
        'En proceso',
        'Resolución 2024',
        'Resolución 2025',
        'Total general'
    ]

    # ✅ TOTAL VERTICAL (fila final)
    total_row = {
        'Año': 'Total general',
        'Oficina de Gestión': '',
        'Tipo de trámite': '',
        'En proceso': t['En proceso'].sum(),
        'Resolución 2024': t['Resolución 2024'].sum(),
        'Resolución 2025': t['Resolución 2025'].sum(),
        'Total general': t['Total general'].sum()
    }

    t = pd.concat([t, pd.DataFrame([total_row])], ignore_index=True)

    return t


# =========================================================
# 6️⃣ FUNCIONES DE GRÁFICAS
# =========================================================

def graficar_mes(df, ruta):

    col_fecha = detectar_columna_fecha(df)

    df = df.copy()
    df[col_fecha] = pd.to_datetime(df[col_fecha], errors='coerce')

    # Crear columna numérica del mes
    df['MES_NUM'] = df[col_fecha].dt.month

    # Diccionario correcto (1–12)
    meses_es = {
        1: 'Enero',
        2: 'Febrero',
        3: 'Marzo',
        4: 'Abril',
        5: 'Mayo',
        6: 'Junio',
        7: 'Julio',
        8: 'Agosto',
        9: 'Septiembre',
        10: 'Octubre',
        11: 'Noviembre',
        12: 'Diciembre'
    }

    # Tabla base agrupada
    t = (
        df.groupby(['MES_NUM','RAD_ANO'])['Numero radicado']
          .nunique()
          .unstack(fill_value=0)
          .reindex(columns=[2024, 2025], fill_value=0)
    )

    # Asegurar que estén los 12 meses
    t = t.reindex(range(1,13), fill_value=0)

    # Reemplazar índice numérico por nombre del mes
    t.index = t.index.map(meses_es)

    # Gráfico
    ax = t.plot(kind='bar')

    #plt.title("Radicados por Mes (2024 vs 2025)")
    plt.xlabel("Mes")
    plt.ylabel("Cantidad")
    plt.legend(title="Año")
    plt.xticks(rotation=45)

    # Etiquetas automáticas
    for container in ax.containers:
        ax.bar_label(container, fontsize=8)

    max_val = t.values.max()
    plt.ylim(0, max_val * 1.15 if max_val > 0 else 1)

    plt.tight_layout()
    plt.savefig(ruta)
    plt.close()

def graficar_dia(df, ruta):

    col_fecha = detectar_columna_fecha(df)

    df = df.copy()
    df[col_fecha] = pd.to_datetime(df[col_fecha], errors='coerce')
    df['DIA_NUM'] = df[col_fecha].dt.weekday

    dias_es = {
        0: 'Lunes',
        1: 'Martes',
        2: 'Miércoles',
        3: 'Jueves',
        4: 'Viernes',
        5: 'Sábado',
        6: 'Domingo'
    }

    df['DIA'] = df['DIA_NUM'].map(dias_es)

    t = (
        df.groupby(['DIA_NUM','DIA','RAD_ANO'])['Numero radicado']
          .nunique()
          .unstack(fill_value=0)
          .reindex(columns=[2024, 2025], fill_value=0)
          .reset_index()
          .sort_values('DIA_NUM')
          .set_index('DIA')
    )

    t = t[[2024, 2025]]

    ax = t.plot(kind='bar')

    #plt.title("Radicados por Día de la Semana (2024 vs 2025)")
    plt.xlabel("Día")
    plt.ylabel("Cantidad")
    plt.legend(title="Año")
    plt.xticks(rotation=45)

    # Etiquetas
    for container in ax.containers:
        ax.bar_label(container, fontsize=8)

    plt.ylim(0, t.values.max() * 1.15 if t.values.max() > 0 else 1)
    plt.tight_layout()
    plt.savefig(ruta)
    plt.close()

# =========================================================
# 7️⃣ GENERADOR DE REPORTES
# =========================================================

from docx.opc.exceptions import PackageNotFoundError
from docx.shared import Inches
from docx import Document
from zipfile import BadZipFile

def insertar_tabla_word(doc, df, titulo):

    doc.add_heading(titulo, level=2)

    tabla = doc.add_table(rows=df.shape[0] + 1, cols=df.shape[1])
    tabla.style = "Table Grid"

    # Encabezados
    for col_idx, col in enumerate(df.columns):
        tabla.rows[0].cells[col_idx].text = str(col)

    # Datos
    for row_idx in range(df.shape[0]):
        for col_idx in range(df.shape[1]):
            tabla.rows[row_idx + 1].cells[col_idx].text = str(df.iloc[row_idx, col_idx])


def generar_sistema_reportes(municipio=None):

    df = pd.read_excel(BASE_EXCEL, sheet_name='CRUDOS')
    df = normalizar_columnas(df)
    df = df[df['RAD_ANO'].isin(ANIOS)]

    municipios = [municipio] if municipio else df['NOM_MUN'].unique()

    os.makedirs(CARPETA_RAIZ, exist_ok=True)

    for mun in municipios:

        df_mun = df[df['NOM_MUN'] == mun].copy()
        if df_mun.empty:
            continue

        base_mun = f"{CARPETA_RAIZ}/{mun}"
        os.makedirs(f"{base_mun}/01_Word", exist_ok=True)
        os.makedirs(f"{base_mun}/02_Graficas", exist_ok=True)

        # ----------------------------
        # Construcción de tablas
        # ----------------------------
        tabla1 = construir_tabla1(df_mun)
        tabla2 = construir_tabla2(df_mun)
        tabla3 = construir_tabla3(df_mun)

        # ----------------------------
        # Generación de gráficas
        # ----------------------------
        ruta_mes = f"{base_mun}/02_Graficas/Radicados_por_mes.png"
        ruta_dia = f"{base_mun}/02_Graficas/Radicados_por_dia.png"

        graficar_mes(df_mun, ruta_mes)
        graficar_dia(df_mun, ruta_dia)

        # ----------------------------
        # Crear o cargar plantilla
        # ----------------------------
        try:
            if not os.path.exists(PLANTILLA_WORD):
                raise FileNotFoundError

            doc = Document(PLANTILLA_WORD)

        except (PackageNotFoundError, FileNotFoundError):
            print("⚠️ Plantilla no encontrada. Se generará documento nuevo.")
            doc = Document()

        # Reemplazar municipio en texto
        for p in doc.paragraphs:
            if "APULO" in p.text:
                p.text = p.text.replace("APULO", mun.upper())

        # ==================================================
        # ORDEN CORRECTO DE INSERCIÓN
        # ==================================================

        doc.add_heading(f"Informe de Trámites – {mun}", level=1)

        # ✔ Tabla 1
        insertar_tabla_word(
            doc,
            tabla1,
            "Tabla 1. Clasificación de los radicados para trámites de conservación catastral"
        )

        # ✔ Tabla 2
        insertar_tabla_word(
            doc,
            tabla2,
            "Tabla 2. Radicados por tipo y quien resuelve"
        )

        # ✔ Gráfica por Mes
        doc.add_heading("Gráfica 1. Radicados por Mes (2024 vs 2025)", level=2)
        doc.add_picture(ruta_mes, width=Inches(5.5))

        # ✔ Gráfica por Día
        doc.add_heading("Gráfica 2. Radicados por Día de la Semana (2024 vs 2025)", level=2)
        doc.add_picture(ruta_dia, width=Inches(5.5))

        # ✔ Tabla 3
        insertar_tabla_word(
            doc,
            tabla3,
            "Tabla 3. Radicados resueltos desagregados por quien resuelve y por año de resolución"
        )

        # ----------------------------
        # Guardar documento
        # ----------------------------
        word_path = f"{base_mun}/01_Word/{mun}_Informe_Final_ACC_2024_2025.docx"
        doc.save(word_path)

        print(f"✅ Reporte completo generado para {mun}")


# =========================================================
# 8️⃣ EJECUCIÓN
# =========================================================

# Municipio específico
generar_sistema_reportes("GUATAVITA")

# Todos los municipios
#generar_sistema_reportes()


✅ Reporte completo generado para GUATAVITA


In [ ]:
from IPython.display import display

df = pd.read_excel(BASE_EXCEL, sheet_name='CRUDOS')
df = normalizar_columnas(df)
df = df[df['RAD_ANO'].isin(ANIOS)]

df_mun = df[df['NOM_MUN'] == "GUATAVITA"]

tabla1 = construir_tabla1(df_mun)
tabla2 = construir_tabla2(df_mun)
tabla3 = construir_tabla3(df_mun)

print("===== TABLA 1 =====")
display(tabla1)

print("===== TABLA 2 =====")
display(tabla2)

print("===== TABLA 3 =====")
display(tabla3)


===== TABLA 1 =====


RAD_ANO,Estado de la radicación,2024,2025,Total general
0,finalizado_por_desistimiento,1,1,2
1,generacion_acto_administrativo,13,33,46
2,informado_notificado_cumplido,687,274,961
3,no_procedente,7,1,8
4,producto_generado,123,211,334
5,radicado,499,619,1118
6,Total general,1330,1139,2469


===== TABLA 2 =====


,Oficina de Gestión,Tipo de trámite,2024,2025,Total general
0,CENTRAL,Cancelación de predios,2,0,2
1,CENTRAL,Carta Catastral Urbano o Rural,3,28,31
2,CENTRAL,Certificado Catastral Especial,48,82,130
3,CENTRAL,Certificado Catastral Nacional,45,28,73
4,CENTRAL,Certificado Plano Predial Catastral,27,73,100
5,CENTRAL,Corrección de propietarios,15,16,31
6,CENTRAL,Mutación de primera - Cambio de propietario,51,63,114
7,CENTRAL,Mutación de quinta - Inscripción de predio,0,4,4
8,CENTRAL,Mutación de segunda - Desenglobe parcial,9,4,13
9,CENTRAL,Mutación de segunda - Desenglobe total,13,8,21


===== TABLA 3 =====


,Año,Oficina de Gestión,Tipo de trámite,En proceso,Resolución 2024,Resolución 2025,Total general
0,2024,CENTRAL,Cancelación de predios,1,0,1,2
1,2024,CENTRAL,Carta Catastral Urbano o Rural,0,3,0,3
2,2024,CENTRAL,Certificado Catastral Especial,0,48,0,48
3,2024,CENTRAL,Certificado Catastral Nacional,0,45,0,45
4,2024,CENTRAL,Certificado Plano Predial Catastral,0,27,0,27
5,2024,CENTRAL,Corrección de propietarios,0,13,2,15
6,2024,CENTRAL,Mutación de primera - Cambio de propietario,1,46,4,51
7,2024,CENTRAL,Mutación de segunda - Desenglobe parcial,7,2,0,9
8,2024,CENTRAL,Mutación de segunda - Desenglobe total,12,0,1,13
9,2024,CENTRAL,Mutación de tercera,2,0,1,3
